## Imports and Defs

In [2]:
import assets.helper as b3
import assets.functions as run

import pandas as pd

## Content

#### Merge Mix and Match all

In [ ]:
try:
    math = run.load_pkl(f'{b3.app_folder}math')
except Exception as e:
    math = run.get_math()
    math = run.save_pkl(math, f'{b3.app_folder}math')


In [ ]:

try:
    setorial = run.load_pkl(f'{b3.app_folder}setorial')
except Exception as e:
    setorial = run.get_classificacao_setorial(setorial='')
    setorial = run.save_pkl(setorial, f'{b3.app_folder}setorial')
# setorial.head(3)

In [ ]:

try:
    b3_cvm = run.load_pkl(f'{b3.app_folder}b3_cvm')
except Exception as e:
    b3_cvm = run.b3_grab(b3.search_url)
    b3_cvm = run.save_pkl(b3_cvm, f'{b3.app_folder}b3_cvm')


In [ ]:
# SETOR: O setor econômico mais amplo ao qual a empresa pertence.
# SUBSETOR: Uma categorização mais específica dentro do setor mais amplo.
# SEGMENTO: Uma classificação ainda mais granular do negócio da empresa.
# DENOM_CIA: Esta é a denominação ou nome da empresa.
# COMPANHIA: Nome ou denominação oficial da empresa listada.
# PREGAO: Refere-se ao nome pelo qual a empresa é conhecida no pregão da bolsa de valores.
# LISTAGEM: Categoria ou segmento de listagem da empresa na bolsa de valores, que pode indicar o nível de governança corporativa ou outros critérios.
# TICK: Abreviação ou símbolo da empresa usada no mercado de ações.
# TICKERS: Símbolos de negociação da empresa em diferentes mercados ou plataformas.
# CD_CVM: Este poderia ser um código ou identificador único relacionado à empresa, possivelmente relacionado à Comissão de Valores Mobiliários do Brasil (CVM).
# CVM: Código ou identificador relacionado à empresa na Comissão de Valores Mobiliários, o órgão regulador do mercado de capitais no Brasil.
# ISIN: Número de Identificação Internacional de Valores Mobiliários – um identificador único para valores mobiliários.
# CNPJ_CIA: Este é o número do Cadastro Nacional da Pessoa Jurídica (CNPJ) da empresa, um identificador único para empresas no Brasil.
# CNPJ: Cadastro Nacional da Pessoa Jurídica – é o número de identificação das empresas brasileiras.
# SITE: Site oficial ou página relevante da empresa.
# ATIVIDADE: Descreve a principal atividade de negócios da empresa.

# ANO: Este é o ano ao qual os dados se referem.
# DT_REFER: Esta é a data de referência para a entrada de dados.
# DT_FIM_EXERC: Esta é a data final para o exercício ou período de relato financeiro.
# DT_INI_EXERC: Esta poderia ser a data inicial para o exercício ou período de relato financeiro.

# AGRUPAMENTO: Isso descreve o nível de agregação dos dados. Por exemplo, 'con' pode indicar dados consolidados.
# BALANCE_SHEET: Isso indica a seção específica da demonstração financeira, como Balanço Patrimonial ('BPA').
# GRUPO_DFP: Isso representa o tipo de grupo de demonstração financeira. Por exemplo, 'DF Consolidado - Balanço Patrimonial Ativo' sugere que é um balanço patrimonial consolidado focado em ativos.
# CD_CONTA: Este poderia ser um código ou identificador único relacionado a uma conta específica ou item de linha na demonstração financeira.
# DS_CONTA: Descreve a conta específica ou item de linha na demonstração financeira, como 'Ativo Total'.

# VL_CONTA: Representa o valor associado à conta específica ou item de linha.
# MOEDA: Isso indica a moeda na qual os valores são representados. 'REAL' sugere Real Brasileiro.
# ESCALA_MOEDA: Isso fornece a escala ou unidade para os valores monetários. 'MIL' pode indicar que os valores estão em milhares.

# ST_CONTA_FIXA: Pode indicar o status ou tipo de conta. O significado de valores como 'S' dependeria do contexto dos dados.
# COLUNA_DF: O propósito desta coluna não é imediatamente claro a partir da amostra. Pode representar algum tipo de classificação ou categorização relacionada aos dados financeiros.

# ESCRITURADOR: Entidade ou empresa responsável por registrar ou gerenciar os valores mobiliários da empresa.
# ACIONISTAS: Informações ou identificadores relacionados aos acionistas da empresa.

# FILENAME: Este é o arquivo de onde os dados são originados. Ele fornece o nome do arquivo que contém a respectiva entrada de dados.
# DEMONSTRATIVO: Este representa o tipo de demonstração financeira. Pode indicar se os dados são de um relatório intermediário (como 'itr') ou de outro tipo de relatório financeiro.
# VERSAO: Isso pode representar a versão ou iteração dos dados/relatórios financeiros.

columns = [
    'SETOR_x',
    'SUBSETOR_x',
    'SEGMENTO_x',
    'DENOM_CIA',
        # 'COMPANHIA',
    'PREGAO',
    'LISTAGEM',
    'TICK',
    'TICKERS',
    'CD_CVM',
        # 'CVM',
        # 'ISIN',
    'CNPJ_CIA',
        # 'CNPJ',
    'SITE',
    'ATIVIDADE',
        # 'ANO',
    'DT_REFER',
        # 'DT_FIM_EXERC',
        # 'DT_INI_EXERC',
    'AGRUPAMENTO',
    'BALANCE_SHEET',
    # 'GRUPO_DFP',
    'CD_CONTA',
    'DS_CONTA',
    'VL_CONTA',
    # 'MOEDA',
    # 'ESCALA_MOEDA',
    # 'ST_CONTA_FIXA',
    # 'COLUNA_DF',
    'ESCRITURADOR',
    'ACIONISTAS', 
    # 'FILENAME', 
    # 'DEMONSTRATIVO', 
    # 'VERSAO',
]


In [ ]:
b3_cvm.keys()

In [ ]:
columns = ['FILENAME', 'DEMONSTRATIVO', 'BALANCE_SHEET', 'ANO', 'AGRUPAMENTO',
       'CNPJ_CIA', 'DT_REFER', 'VERSAO', 'DENOM_CIA', 'CD_CVM', 'GRUPO_DFP',
       'MOEDA', 'ESCALA_MOEDA', 'DT_FIM_EXERC', 'CD_CONTA', 'DS_CONTA',
       'VL_CONTA', 'ST_CONTA_FIXA', 'DT_INI_EXERC', 'COLUNA_DF', 'COMPANHIA',
       'PREGAO', 'TICK', 'LISTAGEM', 'TICKERS', 'ISIN', 
       'ATIVIDADE', 'SETOR', 'SUBSETOR', 'SEGMENTO', 'SITE', 'ESCRITURADOR',]
df = b3_cvm['CONSUMO CICLICO'][columns].set_index('DT_REFER')
df = convert_columns(df)
df['DENOM_CIA'].unique()

#### Acoes


In [101]:
acoes = run.load_pkl(f'{b3.app_folder}acoes')
# Process the data and return the result
acoes['Trimestre'] = pd.to_datetime(acoes['Trimestre'], errors='coerce', dayfirst=True)
acoes['BALANCE_SHEET'] = 'STK'
column_mapping = {
    'Ações ON': '00.01.01',
    'Ações PN': '00.02.01',
    'Ações ON em Tesouraria': '00.01.02',
    'Ações PN em Tesouraria': '00.02.02'
}
acoes = acoes.rename(columns={"Companhia": "DENOM_CIA", "Trimestre": "DT_REFER"})

acoes = acoes.melt(id_vars=['DENOM_CIA', 'DT_REFER', 'BALANCE_SHEET'], 
                        value_vars=['Ações ON', 'Ações PN', 'Ações ON em Tesouraria', 'Ações PN em Tesouraria'],
                        var_name='DS_CONTA', value_name='VL_CONTA').sort_values(by=['DENOM_CIA', 'DT_REFER', 'DS_CONTA'])

acoes['CD_CONTA'] = acoes['DS_CONTA'].map(column_mapping)


In [100]:
intel_b3 = run.load_pkl(f'{b3.app_folder}intel_b3')
df = intel_b3['BENS INDUSTRIAIS']


In [125]:
company = 'ARMAC LOCACAO LOGISTICA E SERVICOS SA'
quarter = '2019-12-31'
mc = acoes['DENOM_CIA'] == company
mc &= acoes['DT_REFER'] == quarter
acoesc = acoes[mc]

mc = df['DENOM_CIA'] == company
mc &= df['DT_REFER'] == quarter
dfc = df[mc]

df_ffill = pd.concat([dfc, acoesc], ignore_index=True).ffill()

In [ ]:
def process_stock_data(group):
    company, quarter = group.name
    
    # Filter acoes based on company and quarter
    mc = acoes['DENOM_CIA'] == company
    mc &= acoes['DT_REFER'] == quarter
    acoesc = acoes[mc]

    # Concatenate and ffill
    return pd.concat([group, acoesc], ignore_index=True).ffill()

result_df


In [130]:

df_ffill

,SETOR,SUBSETOR,SEGMENTO,CNPJ_CIA,DENOM_CIA,CD_CVM,PREGAO,TICK,LISTAGEM,TICKERS,DT_REFER,BALANCE_SHEET,CD_CONTA,DS_CONTA,VL_CONTA
0,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MAQ E EQUIP INDUSTRIAIS,00.242.184/0001-04,ARMAC LOCACAO LOGISTICA E SERVICOS SA,26069,ARMAC,ARML,NM,ARML3,2019-12-31,BPA,01,Ativo Total,116378.0
1,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MAQ E EQUIP INDUSTRIAIS,00.242.184/0001-04,ARMAC LOCACAO LOGISTICA E SERVICOS SA,26069,ARMAC,ARML,NM,ARML3,2019-12-31,BPA,01.01,Ativo Circulante de Curto Prazo,17639.0
2,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MAQ E EQUIP INDUSTRIAIS,00.242.184/0001-04,ARMAC LOCACAO LOGISTICA E SERVICOS SA,26069,ARMAC,ARML,NM,ARML3,2019-12-31,BPA,01.01.01,Caixa e Disponibilidades de Caixa,6793.0
3,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MAQ E EQUIP INDUSTRIAIS,00.242.184/0001-04,ARMAC LOCACAO LOGISTICA E SERVICOS SA,26069,ARMAC,ARML,NM,ARML3,2019-12-31,BPA,01.01.02,Aplicações Financeiras,0.0
4,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MAQ E EQUIP INDUSTRIAIS,00.242.184/0001-04,ARMAC LOCACAO LOGISTICA E SERVICOS SA,26069,ARMAC,ARML,NM,ARML3,2019-12-31,BPA,01.01.02,Aplicações Financeiras,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MAQ E EQUIP INDUSTRIAIS,00.242.184/0001-04,ARMAC LOCACAO LOGISTICA E SERVICOS SA,26069,ARMAC,ARML,NM,ARML3,2019-12-31,DVA,07.08.05,Outros,0.0
463,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MAQ E EQUIP INDUSTRIAIS,00.242.184/0001-04,ARMAC LOCACAO LOGISTICA E SERVICOS SA,26069,ARMAC,ARML,NM,ARML3,2019-12-31,STK,00.01.01,Ações ON,230000.0
464,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MAQ E EQUIP INDUSTRIAIS,00.242.184/0001-04,ARMAC LOCACAO LOGISTICA E SERVICOS SA,26069,ARMAC,ARML,NM,ARML3,2019-12-31,STK,00.01.02,Ações ON em Tesouraria,0.0
465,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MAQ E EQUIP INDUSTRIAIS,00.242.184/0001-04,ARMAC LOCACAO LOGISTICA E SERVICOS SA,26069,ARMAC,ARML,NM,ARML3,2019-12-31,STK,00.02.01,Ações PN,0.0


In [ ]:
intelacoes = {}
for setor, df in intel_b3.items():
    print(setor)
    df_concat = pd.concat([df.set_index(['DENOM_CIA', 'DT_REFER']), acoes.set_index(['DENOM_CIA', 'DT_REFER'])], axis=0, sort=False).reset_index()
    filled_df = df_concat.groupby(['DENOM_CIA', 'DT_REFER'], group_keys=False).apply(lambda group: group.ffill().bfill()).reset_index()
    intelacoes[setor] = filled_df


In [263]:
# Concatenate them vertically
df_concat = pd.concat([df.set_index(['DENOM_CIA', 'DT_REFER']), acoes.set_index(['DENOM_CIA', 'DT_REFER'])], axis=0, sort=False).reset_index()

In [269]:
filled_df = df_concat.groupby(['DENOM_CIA', 'DT_REFER'], group_keys=False).apply(lambda group: group.ffill().bfill()).reset_index()


In [272]:
m = filled_df['DENOM_CIA'] == 'ARMAC LOCACAO LOGISTICA E SERVICOS SA'
m &= filled_df['DT_REFER'] == '2019-12-31'
m &= filled_df['BALANCE_SHEET'] == 'STK'

filled_df[m][cols]

,SETOR,SUBSETOR,SEGMENTO,CNPJ_CIA,DENOM_CIA,CD_CVM,DT_REFER,BALANCE_SHEET,CD_CONTA,DS_CONTA,VL_CONTA
1570850,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MAQ E EQUIP INDUSTRIAIS,00.242.184/0001-04,ARMAC LOCACAO LOGISTICA E SERVICOS SA,26069.0,2019-12-31,STK,00.01.01,Ações ON,230000.0
1570851,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MAQ E EQUIP INDUSTRIAIS,00.242.184/0001-04,ARMAC LOCACAO LOGISTICA E SERVICOS SA,26069.0,2019-12-31,STK,00.01.02,Ações ON em Tesouraria,0.0
1570852,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MAQ E EQUIP INDUSTRIAIS,00.242.184/0001-04,ARMAC LOCACAO LOGISTICA E SERVICOS SA,26069.0,2019-12-31,STK,00.02.01,Ações PN,0.0
1570853,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MAQ E EQUIP INDUSTRIAIS,00.242.184/0001-04,ARMAC LOCACAO LOGISTICA E SERVICOS SA,26069.0,2019-12-31,STK,00.02.02,Ações PN em Tesouraria,0.0


#### Intel


In [ ]:
b3_cvm = run.load_pkl(f'{b3.app_folder}b3_cvm')


In [ ]:
column_types = {'index': 'int',
 'FILENAME': 'category',
 'DEMONSTRATIVO': 'category',
 'BALANCE_SHEET': 'category',
 'ANO': 'category',
 'AGRUPAMENTO': 'category',
 'CNPJ_CIA': 'category',
 'DT_REFER': 'object',
 'VERSAO': 'category',
 'DENOM_CIA': 'category',
 'CD_CVM': 'category',
 'GRUPO_DFP': 'category',
 'MOEDA': 'category',
 'ESCALA_MOEDA': 'category',
 'DT_FIM_EXERC': 'object',
 'CD_CONTA': 'category',
 'DS_CONTA': 'category',
 'VL_CONTA': 'float',
 'ST_CONTA_FIXA': 'category',
 'DT_INI_EXERC': 'object',
 'COLUNA_DF': 'category',
 'COMPANHIA': 'category',
 'PREGAO': 'category',
 'TICK': 'category',
 'LISTAGEM': 'category',
 'CVM': 'category',
 'TICKERS': 'category',
 'ISIN': 'category',
 'CNPJ': 'category',
 'ATIVIDADE': 'category',
 'SETOR': 'category',
 'SUBSETOR': 'category',
 'SEGMENTO': 'category',
 'SITE': 'category',
 'ESCRITURADOR': 'category',
 'CD_CONTA_original': 'category',
 'DS_CONTA_original': 'category'}
date_columns = ['DT_REFER', 'DT_FIM_EXERC', 'DT_INI_EXERC']


In [ ]:
df = pd.read_csv('BENS INDUSTRIAIS_intel.csv', dtype=column_types, index_col='Unnamed: 0', parse_dates=True)
# df = pd.read_csv('COMUNICACOES_df.csv')
# df_typed = pd.read_csv(f)


In [ ]:
# Criando um dicionário onde as chaves são os nomes das colunas e os valores são os valores únicos para cada coluna
col_dict = {col: df[col].unique().tolist() for col in df.columns}
df.columns

In [ ]:
col_dict['BALANCE_SHEET']

In [ ]:
df = df[['CNPJ_CIA', 'DENOM_CIA', 'DT_REFER', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA', 'SETOR', 'SUBSETOR', 'SEGMENTO', 'DT_INI_EXERC', 'ATIVIDADE', 'SITE', 'ESCRITURADOR', 'FILENAME', 'DEMONSTRATIVO', 'BALANCE_SHEET', 'ANO', 'AGRUPAMENTO', 'VERSAO', 'GRUPO_DFP', 'MOEDA', 'ESCALA_MOEDA', 'ST_CONTA_FIXA', 'COLUNA_DF', 'COMPANHIA', 'TICKERS', 'CVM', 'ISIN', 'DT_FIM_EXERC', ]]
# 'PREGAO', 'TICK', 'CD_CVM', 'LISTAGEM', # 'CD_CONTA_original', 'DS_CONTA_original', 
df

In [ ]:
formulas_old = [
    # Relações Entre Ativos e Passivos
    ('_020302_reservas_de_capital', '_020303_reservas_de_reavaliacao', '_020304_reservas_de_lucros'),
    # Dívida
    ('_0201040101_emprestimos_e_financiamentos_em_moeda_nacional', '_0201040102_emprestimos_e_financiamentos_em_moeda_estrangeira', '_02010402_debentures', '_02010403_arrendamentos', '_02010409_outros_emprestimos_financiamentos_e_debentures'),
    ('_0202010101_emprestimos_e_financiamentos_em_moeda_nacional', '_0202010102_emprestimos_e_financiamentos_em_moeda_estrangeira', '_02020102_debentures', '_02020103_arrendamentos', '_02020209_outros_emprestimos_financiamentos_e_debentures'),
    ('_0201040102_emprestimos_e_financiamentos_em_moeda_estrangeira', '_0202010102_emprestimos_e_financiamentos_em_moeda_estrangeira'),
    ('_010101_caixa_e_disponibilidades_de_caixa',),
    ('_010202_investimentos_nao_capex', '_010203_imobilizados', '_010204_intangivel'),
    ('_0305_lajir_ebit_resultado_antes_do_resultado_financeiro_e_dos_tributos', '_070401_depreciacao_e_amortizacao'),
    # Resultados Fundamentalistas
    ('_0203_patrimonio_liquido',),
    ('_010101_caixa_e_disponibilidades_de_caixa',),
    ('_070803_remuneracao_de_capital_de_terceiros', '_070804_remuneracao_de_capital_proprio'),
    # Análise do Fluxo de Caixa
    ('_0601_caixa_das_operacoes', '_0602_caixa_de_investimentos_capex'),
    ('_0603_caixa_de_financiamento',),
    ('_060201_investimentos', '_060202_imobilizado_e_intangivel'),
]

def de_transform_corrected(key):
    # Strip the leading underscore and split at the first underscore
    parts = key[1:].split('_', 1)

    # Adjust code by inserting periods every two characters
    code = '.'.join([parts[0][i:i+2] for i in range(0, len(parts[0]), 2)])
    
    # Adjust description capitalization
    description = ' '.join([word.capitalize() if word not in ['e', 'de', 'do', 'dos', 'da', 'das', 'em'] else word.lower() for word in parts[1].split('_')])
    
    return code, description

# De-transform the formulas using the corrected function
formulas = []
for group in formulas_old:
    new_group = []
    for key in group:
        new_group.append(de_transform_corrected(key))
    formulas.append(tuple(new_group))

formulas


In [ ]:
balance_sheet = df.groupby(['CNPJ_CIA', 'DT_REFER'], group_keys=True)
balance_sheet.groups.keys()
df_ = balance_sheet.get_group(('00.242.184/0001-04', '2019-12-31'))


In [ ]:
# (, 'Emprestimos e Financiamentos em Moeda Estrangeira'),
m = df['CD_CONTA'] == '07.08.04'
df[m]

##### Dados Abertos

In [ ]:
def read(file):
    path = "C:\\Users\\faust\\OneDrive\\Área de Trabalho\\dados abertos\\"
    df = pd.read_csv(path+file+".csv", sep=';', encoding='latin1')
    return df.head(25)



In [ ]:
file = "fca_cia_aberta_2010"
fca_cia_aberta_2010 = read(file)
# link para o NSD do formulário cadastral
fca_cia_aberta_2010

In [ ]:
file = "fca_cia_aberta_auditor_2010"
fca_cia_aberta_auditor_2010 = read(file)
# informações dos auditores CNPJ e CPF, datas dos auditores CPF
fca_cia_aberta_auditor_2010

In [ ]:
file = "fca_cia_aberta_canal_divulgacao_2010"
fca_cia_aberta_canal_divulgacao_2010 = read(file)
# Onde as DRE são divulgadas
fca_cia_aberta_canal_divulgacao_2010

In [ ]:
file = "fca_cia_aberta_departamento_acionistas_2010"
fca_cia_aberta_departamento_acionistas_2010 = read(file)
# Endereços dos DRI
fca_cia_aberta_departamento_acionistas_2010

In [ ]:
file = "fca_cia_aberta_dri_2010"
fca_cia_aberta_dri_2010 = read(file)
# NOMES e endereços dos DRI
fca_cia_aberta_dri_2010

In [ ]:
file = "fca_cia_aberta_endereco_2010"
fca_cia_aberta_endereco_2010 = read(file)
# Endereço completo do DRI
fca_cia_aberta_endereco_2010

In [ ]:
file = "fca_cia_aberta_geral_2010"
fca_cia_aberta_geral_2010 = read(file)
# Cadastro CVM, Atividade, Descrição e Controle Acionário
fca_cia_aberta_geral_2010

In [ ]:
file = "fca_cia_aberta_pais_estrangeiro_negociacao_2010"
fca_cia_aberta_pais_estrangeiro_negociacao_2010 = read(file)
# País estrangeiro... ?
fca_cia_aberta_pais_estrangeiro_negociacao_2010

In [ ]:
file = "fca_cia_aberta_valor_mobiliario_2010"
fca_cia_aberta_valor_mobiliario_2010 = read(file)
# Valor mobiliário, Mercado e Segmento
fca_cia_aberta_valor_mobiliario_2010

In [ ]:
file = "fre_cia_aberta_2023"
fre_cia_aberta_2023 = read(file)
# Link do Documento
fre_cia_aberta_2023

In [ ]:
file = "fre_cia_aberta_acao_entregue_2023"
fre_cia_aberta_acao_entregue_2023 = read(file)
# Remuneração da Diretoria
fre_cia_aberta_acao_entregue_2023

In [ ]:
file = "fre_cia_aberta_administrador_declaracao_genero_2023"
fre_cia_aberta_administrador_declaracao_genero_2023 = read(file)
# Gênero dos administradores
fre_cia_aberta_administrador_declaracao_genero_2023

In [ ]:
file = "fre_cia_aberta_administrador_declaracao_raca_2023"
fre_cia_aberta_administrador_declaracao_raca_2023 = read(file)
# Raça dos administradores
fre_cia_aberta_administrador_declaracao_raca_2023

In [ ]:
file = "fre_cia_aberta_administrador_membro_conselho_fiscal_2023"
fre_cia_aberta_administrador_membro_conselho_fiscal_2023 = read(file)
# Membros do Conselho Fiscal
fre_cia_aberta_administrador_membro_conselho_fiscal_2023

In [ ]:
file = "fre_cia_aberta_ativo_imobilizado_2023"
fre_cia_aberta_ativo_imobilizado_2023 = read(file)
# Ativos e Propriedades por empresa
fre_cia_aberta_ativo_imobilizado_2023

In [ ]:
file = "fre_cia_aberta_ativo_intangivel_2023"
fre_cia_aberta_ativo_intangivel_2023 = read(file)
# Ativos e Propriedades por empresa
fre_cia_aberta_ativo_intangivel_2023

In [ ]:
file = "fre_cia_aberta_auditor_2023"
fre_cia_aberta_auditor_2023 = read(file)
# Remuneração por auditor
fre_cia_aberta_auditor_2023

In [ ]:
file = "fre_cia_aberta_auditor_responsavel_2023"
fre_cia_aberta_auditor_responsavel_2023 = read(file)
# Endereço do Auditor
fre_cia_aberta_auditor_responsavel_2023

In [ ]:
file = "fre_cia_aberta_capital_social_2023"
fre_cia_aberta_capital_social_2023 = read(file)
# Modificações no Capital Social e Ações
fre_cia_aberta_capital_social_2023

In [ ]:
file = "fre_cia_aberta_capital_social_aumento_2023"
fre_cia_aberta_capital_social_aumento_2023 = read(file)
# Idem
fre_cia_aberta_capital_social_aumento_2023

In [ ]:
file = "fre_cia_aberta_capital_social_aumento_classe_acao_2023"
fre_cia_aberta_capital_social_aumento_classe_acao_2023 = read(file)
# Em branco
fre_cia_aberta_capital_social_aumento_classe_acao_2023

In [ ]:
file = "fre_cia_aberta_capital_social_classe_acao_2023"
fre_cia_aberta_capital_social_classe_acao_2023 = read(file)
# Preferencial Classe A, B e C
fre_cia_aberta_capital_social_classe_acao_2023

In [ ]:
file = "fre_cia_aberta_capital_social_desdobramento_2023"
fre_cia_aberta_capital_social_desdobramento_2023 = read(file)
# Desdobramentos de Ações
fre_cia_aberta_capital_social_desdobramento_2023

In [ ]:
file = "fre_cia_aberta_capital_social_desdobramento_classe_acao_2023"
fre_cia_aberta_capital_social_desdobramento_classe_acao_2023 = read(file)
# em branco
fre_cia_aberta_capital_social_desdobramento_classe_acao_2023

In [ ]:
file = "fre_cia_aberta_capital_social_reducao_2023"
fre_cia_aberta_capital_social_reducao_2023 = read(file)
# Redução de capital
fre_cia_aberta_capital_social_reducao_2023

In [ ]:
file = "fre_cia_aberta_capital_social_titulo_conversivel_2023"
fre_cia_aberta_capital_social_titulo_conversivel_2023 = read(file)
# Redução de capital
fre_cia_aberta_capital_social_titulo_conversivel_2023

In [ ]:
file = "fre_cia_aberta_direito_acao_2023"
fre_cia_aberta_direito_acao_2023 = read(file)
# ?
fre_cia_aberta_direito_acao_2023

In [ ]:
file = "fre_cia_aberta_historico_emissor_2023"
fre_cia_aberta_historico_emissor_2023 = read(file)
# Constituição do emissor e local
fre_cia_aberta_historico_emissor_2023

In [ ]:
file = "fre_cia_aberta_informacao_financeira_2023"
fre_cia_aberta_informacao_financeira_2023 = read(file)
# DRE Resumido
fre_cia_aberta_informacao_financeira_2023

In [ ]:
file = "fre_cia_aberta_membro_comite_2023"
fre_cia_aberta_membro_comite_2023 = read(file)
# CPF e Remuneração
fre_cia_aberta_membro_comite_2023

In [ ]:
file = "fre_cia_aberta_obrigacao_2023"
fre_cia_aberta_obrigacao_2023 = read(file)
# Obrigações e Dívidas
fre_cia_aberta_obrigacao_2023

In [ ]:
file = "fre_cia_aberta_outro_valor_mobiliario_2023"
fre_cia_aberta_outro_valor_mobiliario_2023 = read(file)
# ?
fre_cia_aberta_outro_valor_mobiliario_2023

In [ ]:
file = "fre_cia_aberta_participacao_sociedade_2023"
fre_cia_aberta_participacao_sociedade_2023 = read(file)
# Participações em outras empresas
fre_cia_aberta_participacao_sociedade_2023

In [ ]:
file = "fre_cia_aberta_participacao_sociedade_valorizacao_acao_2023"
fre_cia_aberta_participacao_sociedade_valorizacao_acao_2023 = read(file)
# ?
fre_cia_aberta_participacao_sociedade_valorizacao_acao_2023

In [ ]:
file = "fre_cia_aberta_posicao_acionaria_2023"
fre_cia_aberta_posicao_acionaria_2023 = read(file)
# Composição acionária por acionista majoritário
fre_cia_aberta_posicao_acionaria_2023[['CNPJ_Companhia', 'Data_Referencia', 'Versao', 'ID_Documento',
       'ID_Acionista', 'Acionista', 'Tipo_Pessoa_Acionista',
       'CPF_CNPJ_Acionista', 'ID_Acionista_Relacionado',
       'Acionista_Relacionado', 'Tipo_Pessoa_Acionista_Relacionado',
       'CPF_CNPJ_Acionista_Relacionado',
       'Quantidade_Acao_Ordinaria_Circulacao',
       'Percentual_Acao_Ordinaria_Circulacao',
       'Quantidade_Acao_Preferencial_Circulacao',
       'Percentual_Acao_Preferencial_Circulacao',
       'Quantidade_Total_Acoes_Circulacao',
       'Percentual_Total_Acoes_Circulacao', ]]
# fre_cia_aberta_posicao_acionaria_2023.columns


In [ ]:
file = "fre_cia_aberta_relacao_familiar_2023"
fre_cia_aberta_relacao_familiar_2023 = read(file)
# Parentescos
fre_cia_aberta_relacao_familiar_2023

In [ ]:
file = "fre_cia_aberta_relacao_subordinacao_2023"
fre_cia_aberta_relacao_subordinacao_2023 = read(file)
# Subordnicação 
fre_cia_aberta_relacao_subordinacao_2023

In [ ]:
file = "fre_cia_aberta_transacao_parte_relacionada_2023"
fre_cia_aberta_transacao_parte_relacionada_2023 = read(file)
# Partes relacionadas
fre_cia_aberta_transacao_parte_relacionada_2023

In [ ]:
file = "fre_cia_aberta_2023"
fre_cia_aberta_2023 = read(file)
# Link do Documento
fre_cia_aberta_2023

In [ ]:
def plot_group(cias_por_setor, window):
    for company, group_df in cias_por_setor:
        try:
            # Calculate the moving average for the last 4 periods
            group_df['MA'] = group_df['VL_CONTA'].rolling(window=window).mean()
            
            # Calculate the rolling sum for the last 4 periods
            group_df['Rolling_Sum'] = group_df['VL_CONTA'].rolling(window=window).sum()
            
            # Calculate the lifelong cumulative sum
            group_df['Cumulative_Sum'] = group_df['VL_CONTA'].cumsum()
            
            # Plot raw data
            # group_df['VL_CONTA'].plot(label='Raw Data', legend=True)

            # Plot moving average
            group_df['MA'].plot(label=f'{window} Quarters Moving Average', legend=True, linestyle='--')
            
            # Plot rolling sum
            group_df['Rolling_Sum'].plot(label=f'{window} Quarters Sum', legend=True, linestyle='-.')
            
            # Plot lifelong cumulative sum
            group_df['Cumulative_Sum'].plot(label='Lifelong Cumulative Sum', legend=True, linestyle='-.')

            plt.title(f"{group_df['CD_CVM'].iloc[-1]} {group_df['DENOM_CIA'].iloc[-1]}")
            plt.show()
        except Exception as e:
            print(f"Error plotting for {company}: {e}")
    return True

cias_por_setor = df[(df['AGRUPAMENTO'] == 'con') & (df['CD_CONTA'] == '3.11')].groupby('CD_CVM')
window = 4*1
plot_group(cias_por_setor, window)

In [ ]:
import matplotlib.pyplot as plt

cias_por_setor = df[(df['AGRUPAMENTO'] == 'con') & (df['CD_CONTA'] == '2.03')].groupby('DENOM_CIA')

fig, ax = plt.subplots(figsize=(10, 6))

for company, group_df in cias_por_setor:
    try:
        group_df[['VL_CONTA']].plot(ax=ax, label=company)
    except:
        print(company)

ax.set_title("VL_CONTA by Company")
ax.set_ylabel("VL_CONTA")
ax.set_xlabel("Index")
ax.legend(loc="best")

plt.show()


#### Intelacoes Fundamentalista

In [ ]:
intelacoes = run.load_pkl(f'{b3.app_folder}intelacoes')
intelacoes.keys()

In [37]:
df = pd.read_pickle('BENS INDUSTRIAIS_intelacoes.pkl')
# df.to_csv('BENS INDUSTRIAIS_intelacoes.csv')

In [26]:
columns = ['Unnamed: 0', 'FILENAME', 'DEMONSTRATIVO', 'BALANCE_SHEET', 'ANO',
       'AGRUPAMENTO', 'CNPJ_CIA', 'DT_REFER', 'VERSAO', 'DENOM_CIA', 'CD_CVM',
       'GRUPO_DFP', 'MOEDA', 'ESCALA_MOEDA', 'DT_FIM_EXERC', 'CD_CONTA',
       'DS_CONTA', 'VL_CONTA', 'ST_CONTA_FIXA', 'DT_INI_EXERC', 'COLUNA_DF',
       'COMPANHIA', 'PREGAO', 'TICK', 'LISTAGEM', 'CVM', 'TICKERS', 'ISIN',
       'CNPJ', 'ATIVIDADE', 'SETOR', 'SUBSETOR', 'SEGMENTO', 'SITE',
       'ESCRITURADOR', 'CD_CONTA_original', 'DS_CONTA_original', 'Companhia',
       'Trimestre', 'Ações ON', 'Ações PN', 'Ações ON em Tesouraria',
       'Ações PN em Tesouraria', 'URL']
cols = ['SETOR', 'SUBSETOR', 'SEGMENTO', 'CNPJ_CIA', 'DENOM_CIA', 'CD_CVM',  'PREGAO', 'TICK', 'LISTAGEM', 'TICKERS', 'DT_REFER', 'BALANCE_SHEET', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA', ]
datetime_cols = ['DT_REFER', 'DT_FIM_EXERC', 'DT_INI_EXERC', 'Trimestre']
company_cols = ['SETOR', 'SUBSETOR', 'SEGMENTO', 'CNPJ_CIA', 'DENOM_CIA', 'CD_CVM']
dateseries_col = ['DT_REFER']
sheet_cols = ['BALANCE_SHEET', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA']
stocks_cols = ['Ações ON', 'Ações PN', 'Ações ON em Tesouraria', 'Ações PN em Tesouraria']


In [38]:
# Grouping by 'SETOR', 'SUBSETOR', and 'SEGMENTO' to count unique companies and aggregate their TICK values
ticks_by_setor = df.groupby(['SETOR', 'SUBSETOR', 'SEGMENTO']).agg({
    'DENOM_CIA': 'nunique',
    'TICK': lambda x: list(set(x.dropna()))
}).reset_index()

# Renaming columns for clarity
ticks_by_setor.rename(columns={'DENOM_CIA': 'TOTAL DE COMPANHIAS'}, inplace=True)

ticks_by_setor


,SETOR,SUBSETOR,SEGMENTO,TOTAL DE COMPANHIAS,TICK
0,BENS INDUSTRIAIS,COMERCIO,MATERIAL DE TRANSPORTE,4,"[EPAR, WLMM, MTRO, MMAQ]"
1,BENS INDUSTRIAIS,CONSTRUCAO E ENGENHARIA,CONSTRUCAO PESADA,1,[AZEV]
2,BENS INDUSTRIAIS,CONSTRUCAO E ENGENHARIA,ENGENHARIA CONSULTIVA,1,[SOND]
3,BENS INDUSTRIAIS,CONSTRUCAO E ENGENHARIA,PRODUTOS PARA CONSTRUCAO,3,"[HAGA, PTBL, ETER]"
4,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,ARMAS E MUNICOES,1,[TASA]
5,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MAQ E EQUIP CONSTRUCAO E AGRICOLAS,1,[MTSA]
6,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MAQ E EQUIP INDUSTRIAIS,15,"[MILS, ARML, ROMI, AERI, KEPL, FRIO, NORD, BDL..."
7,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MOTORES COMPRESSORES E OUTROS,2,"[SHUL, WEGE]"
8,BENS INDUSTRIAIS,MATERIAL DE TRANSPORTE,MATERIAL AERONAUTICO E DE DEFESA,1,[EMBR]
9,BENS INDUSTRIAIS,MATERIAL DE TRANSPORTE,MATERIAL RODOVIARIO,7,"[FRAS, RSUL, RAPT, MWET, POMO, RCSL]"


In [95]:
conta = '02.03'

# Filtering the data for the account 'CD_CONTA' with value '02.03' and the latest quarter
conta_by_setor = df[df['DT_REFER'] == df['DT_REFER'].max()]
conta_by_setor = conta_by_setor[conta_by_setor['CD_CONTA'] == conta]

# Grouping by 'SETOR', 'SUBSETOR', and 'SEGMENTO' to aggregate 'VL_CONTA' values
conta_by_setor = conta_by_setor.groupby(['SETOR', 'SUBSETOR', 'SEGMENTO', ]).agg({
    'VL_CONTA': ['nunique', 'sum', 'max', 'min', 'mean', 'std', 'skew', lambda x: x.kurt()]
}).reset_index()

conta_by_setor.rename(columns={'VL_CONTA': conta}, inplace=True)

conta_by_setor


SETOR                 SUBSETOR  \
                                                
0   BENS INDUSTRIAIS                 COMERCIO   
1   BENS INDUSTRIAIS  CONSTRUCAO E ENGENHARIA   
2   BENS INDUSTRIAIS  CONSTRUCAO E ENGENHARIA   
3   BENS INDUSTRIAIS  CONSTRUCAO E ENGENHARIA   
4   BENS INDUSTRIAIS  MAQUINAS E EQUIPAMENTOS   
5   BENS INDUSTRIAIS  MAQUINAS E EQUIPAMENTOS   
6   BENS INDUSTRIAIS  MAQUINAS E EQUIPAMENTOS   
7   BENS INDUSTRIAIS  MAQUINAS E EQUIPAMENTOS   
8   BENS INDUSTRIAIS   MATERIAL DE TRANSPORTE   
9   BENS INDUSTRIAIS   MATERIAL DE TRANSPORTE   
10  BENS INDUSTRIAIS                 SERVICOS   
11  BENS INDUSTRIAIS               TRANSPORTE   
12  BENS INDUSTRIAIS               TRANSPORTE   
13  BENS INDUSTRIAIS               TRANSPORTE   
14  BENS INDUSTRIAIS               TRANSPORTE   
15  BENS INDUSTRIAIS               TRANSPORTE   
16  BENS INDUSTRIAIS               TRANSPORTE   

                              SEGMENTO   02.03                            \
                                       nunique          sum          max   
0               MATERIAL DE TRANSPORTE       4   36982305.0   35946564.0   
1                    CONSTRUCAO PESADA       1     119223.0     119223.0   
2                ENGENHARIA CONSULTIVA       1      55650.0      55650.0   
3             PRODUTOS PARA CONSTRUCAO       3  -54385455.0     694225.0   
4                     ARMAS E MUNICOES       1    1058535.0    1058535.0   
5   MAQ E EQUIP CONSTRUCAO E AGRICOLAS       1  417586792.0  417586792.0   
6              MAQ E EQUIP INDUSTRIAIS      11    3028259.0    1339211.0   
7       MOTORES  COMPRESSORES E OUTROS       2   17026419.0   15830523.0   
8     MATERIAL AERONAUTICO E DE DEFESA       1   13286295.0   13286295.0   
9                  MATERIAL RODOVIARIO       6    9464561.0    4113628.0   
10                   SERVICOS DIVERSOS       8    5120911.0    2814565.0   
11              EXPLORACAO DE RODOVIAS      15   31817912.0   12664433.0   
12     SERVICOS DE APOIO E ARMAZENAGEM       7     439701.0    2278855.0   
13                    TRANSPORTE AEREO       2  -40097302.0  -19682000.0   
14              TRANSPORTE FERROVIARIO       5   33988706.0   15643512.0   
15              TRANSPORTE HIDROVIARIO       3    2546529.0    1490235.0   
16               TRANSPORTE RODOVIARIO       2    2434782.0    1632482.0   

                                                                  
            min          mean           std      skew <lambda_0>  
0      135207.0  9.245576e+06  1.780205e+07  1.999062   3.996845  
1      119223.0  1.192230e+05           NaN       NaN        NaN  
2       55650.0  5.565000e+04           NaN       NaN        NaN  
3   -55453625.0 -1.812848e+07  3.232492e+07 -1.731860        NaN  
4     1058535.0  1.058535e+06           NaN       NaN        NaN  
5   417586792.0  4.175868e+08           NaN       NaN        NaN  
6    -1776682.0  2.752963e+05  9.001406e+05 -1.051877   1.554481  
7     1195896.0  8.513210e+06  1.034824e+07       NaN        NaN  
8    13286295.0  1.328630e+07           NaN       NaN        NaN  
9      -13547.0  1.577427e+06  1.816699e+06  0.529987  -1.969247  
10       -368.0  6.401139e+05  9.805898e+05  1.962739   3.671185  
11    -847591.0  2.121194e+06  3.343823e+06  2.499325   7.294285  
12   -3221563.0  6.281443e+04  2.422020e+06 -0.623126  -1.482176  
13  -20415302.0 -2.004865e+07  5.185228e+05       NaN        NaN  
14    1908897.0  6.797741e+06  5.190763e+06  1.688919   3.547946  
15     116220.0  8.488430e+05  6.915357e+05 -0.583331        NaN  
16     802300.0  1.217391e+06  5.870273e+05       NaN        NaN

In [99]:
company = 'ARMAC LOCACAO LOGISTICA E SERVICOS SA'
quarter = '2020-12-31'
dfc = df
mask = df['DENOM_CIA'] == company


dfc[mask][cols].drop_duplicates()

,SETOR,SUBSETOR,SEGMENTO,CNPJ_CIA,DENOM_CIA,CD_CVM,PREGAO,TICK,LISTAGEM,TICKERS,DT_REFER,BALANCE_SHEET,CD_CONTA,DS_CONTA,VL_CONTA
0,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MAQ E EQUIP INDUSTRIAIS,00.242.184/0001-04,ARMAC LOCACAO LOGISTICA E SERVICOS SA,26069.0,ARMAC,ARML,NM,ARML3,2019-12-31,BPA,01,Ativo Total,116378.0
1,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MAQ E EQUIP INDUSTRIAIS,00.242.184/0001-04,ARMAC LOCACAO LOGISTICA E SERVICOS SA,26069.0,ARMAC,ARML,NM,ARML3,2020-12-31,BPA,01,Ativo Total,430028.0
2,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MAQ E EQUIP INDUSTRIAIS,00.242.184/0001-04,ARMAC LOCACAO LOGISTICA E SERVICOS SA,26069.0,ARMAC,ARML,NM,ARML3,2021-03-31,BPA,01,Ativo Total,599001.0
3,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MAQ E EQUIP INDUSTRIAIS,00.242.184/0001-04,ARMAC LOCACAO LOGISTICA E SERVICOS SA,26069.0,ARMAC,ARML,NM,ARML3,2021-06-30,BPA,01,Ativo Total,847490.0
4,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MAQ E EQUIP INDUSTRIAIS,00.242.184/0001-04,ARMAC LOCACAO LOGISTICA E SERVICOS SA,26069.0,ARMAC,ARML,NM,ARML3,2021-09-30,BPA,01,Ativo Total,1958982.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1570885,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MAQ E EQUIP INDUSTRIAIS,00.242.184/0001-04,ARMAC LOCACAO LOGISTICA E SERVICOS SA,26069.0,ARMAC,ARML,NM,ARML3,2022-09-30,STK,00.02.02,Ações PN em Tesouraria,0.0
1570886,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MAQ E EQUIP INDUSTRIAIS,00.242.184/0001-04,ARMAC LOCACAO LOGISTICA E SERVICOS SA,26069.0,ARMAC,ARML,NM,ARML3,2022-12-31,STK,00.01.01,Ações ON,346016041.0
1570887,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MAQ E EQUIP INDUSTRIAIS,00.242.184/0001-04,ARMAC LOCACAO LOGISTICA E SERVICOS SA,26069.0,ARMAC,ARML,NM,ARML3,2022-12-31,STK,00.01.02,Ações ON em Tesouraria,686800.0
1570888,BENS INDUSTRIAIS,MAQUINAS E EQUIPAMENTOS,MAQ E EQUIP INDUSTRIAIS,00.242.184/0001-04,ARMAC LOCACAO LOGISTICA E SERVICOS SA,26069.0,ARMAC,ARML,NM,ARML3,2022-12-31,STK,00.02.01,Ações PN,0.0
